In [1]:
look_back = 60
file_modello = "LSTM.keras"

In [2]:
import pandas as pd
import yfinance as yf
import funzioni as fx
import plotly.offline as pyo
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.models import load_model
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os

pd.set_option("display.max_columns", None)

def visualizza_grafico(ticker):
    import plotly.graph_objs as go
    fig = fx.grafico_base(ticker)
    tg = go.Scatter(
        x = ticker[ticker['Target'] == True].index,
        y = ticker[ticker['Target'] == True]['Close'],
        mode = 'markers',
        marker = dict(
            size = 15,
            color = 'rgba(0, 250, 0, .9)'
        ),
        name = 'target'
    )
    fig.add_trace(tg, row=1, col=1)
    pyo.plot(fig, filename="prova.html", auto_open=True)

def addestramento(model, features, target):    
    X, Y = fx.converti_in_XY(features, target, look_back)
    Y = np.where(Y, 1, 0)
    class_weights = compute_class_weight('balanced', classes=np.unique(Y), y=Y)
    class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
    print(class_weights) 

    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    if model == 0:
        model = keras.Sequential([
            layers.LSTM(200, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
            layers.Dropout(0.2),
            layers.LSTM(200, return_sequences=True),  # Nuovo layer LSTM 1
            layers.Dropout(0.2),
            layers.LSTM(200, return_sequences=True),  # Nuovo layer LSTM 2
            layers.Dropout(0.2),
            layers.LSTM(200),  # Ultimo layer LSTM
            layers.Dropout(0.2),
            layers.Dense(1, activation='sigmoid')
        ])
    custom_optimizer = Adam(learning_rate=0.0001)

    model.compile(
        optimizer=custom_optimizer,
        loss='binary_crossentropy',
        metrics=[
            Precision(name='precision'),
            Recall(name='recall'),
            AUC(name='auc'),
            'accuracy'
        ]
    )

    scaler = StandardScaler()
    n_samples, n_timesteps, n_features = X.shape
    X_reshape = X.reshape((n_samples, n_timesteps * n_features))
    X_scaled = scaler.fit_transform(X_reshape)
    X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)
    X_train_scaled = X_train.reshape((-1, n_timesteps, n_features))
    X_test_scaled = X_test.reshape((-1, n_timesteps, n_features))
    
    model.fit(
        X_train_scaled, Y_train,
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        validation_data=(X_test_scaled, Y_test),
        callbacks=[
            early_stopping
        ]
    )

    loss, precision, recall, auc, accuracy = model.evaluate(X_test_scaled, Y_test)
    print(f"\033[42mLoss: {loss}, Precision: {precision}, Recall: {recall}, AUC: {auc}, Accuracy: {accuracy}\033[0m")
    with open("result.txt", "a") as f:
        f.write(f"Loss: {loss}, Precision: {precision}, Recall: {recall}, AUC: {auc}, Accuracy: {accuracy}\n")
    return model, scaler



In [3]:
elenco_usin = pd.read_excel("DeGiro con formule.xlsx")
elenco_usin = elenco_usin.loc[elenco_usin["Categoria"] != "D", ["USIN", "Ticker"]]
elenco_usin = elenco_usin.sample(frac=1)
tot_usin = len(elenco_usin)
i = 1
if os.path.exists(file_modello):
    model = load_model(file_modello)
else:
    model = 0
for index, usin in elenco_usin.iterrows():       
    print(f"\r{i} di {tot_usin}) Caricamento ticker {usin['Ticker']}                           ", end=' ', flush=True)
    ticker = yf.download(usin["USIN"], start='2005-01-01', end='2023-07-31', progress=False)
    if len(ticker) < 500:
        continue
    else:
        ticker = fx.crea_indicatori(ticker)
        features, target = fx.features_e_target(ticker)
        ticker["Target"] = target
        model, scaler = addestramento(model, features=features, target=target) 
        model.save(file_modello)
        i += 1

print("fine addestramento")

1 di 2581) Caricamento ticker BATRK                            [ 0.51112485 22.97222222]
Epoch 1/100
42/42 [==============================] - 39s 552ms/step - loss: 0.6035 - precision: 0.0391 - recall: 0.7037 - auc: 0.7266 - accuracy: 0.6410 - val_loss: 0.6189 - val_precision: 0.0870 - val_recall: 0.8889 - val_auc: 0.8887 - val_accuracy: 0.7432
Epoch 2/100
42/42 [==============================] - 20s 477ms/step - loss: 0.4025 - precision: 0.0754 - recall: 0.8519 - auc: 0.9188 - accuracy: 0.7838 - val_loss: 0.2320 - val_precision: 0.2121 - val_recall: 0.7778 - val_auc: 0.9425 - val_accuracy: 0.9154
Epoch 3/100
42/42 [==============================] - 20s 488ms/step - loss: 0.3292 - precision: 0.1420 - recall: 0.8519 - auc: 0.9287 - accuracy: 0.8919 - val_loss: 0.3300 - val_precision: 0.1481 - val_recall: 0.8889 - val_auc: 0.9331 - val_accuracy: 0.8580
Epoch 4/100
42/42 [==============================] - 20s 477ms/step - loss: 0.2216 - precision: 0.1287 - recall: 0.9630 - auc: 0.9659 - a

42/42 [==============================] - 20s 477ms/step - loss: 0.0941 - precision: 0.2571 - recall: 1.0000 - auc: 0.9887 - accuracy: 0.9410 - val_loss: 0.1320 - val_precision: 0.3810 - val_recall: 0.8889 - val_auc: 0.9312 - val_accuracy: 0.9577
Epoch 33/100
42/42 [==============================] - 20s 489ms/step - loss: 0.0877 - precision: 0.2842 - recall: 1.0000 - auc: 0.9884 - accuracy: 0.9486 - val_loss: 0.1239 - val_precision: 0.4211 - val_recall: 0.8889 - val_auc: 0.9286 - val_accuracy: 0.9637
Epoch 34/100
42/42 [==============================] - 21s 494ms/step - loss: 0.1002 - precision: 0.2755 - recall: 1.0000 - auc: 0.9845 - accuracy: 0.9463 - val_loss: 0.1451 - val_precision: 0.3478 - val_recall: 0.8889 - val_auc: 0.9318 - val_accuracy: 0.9517
Epoch 35/100
42/42 [==============================] - 20s 480ms/step - loss: 0.0959 - precision: 0.2842 - recall: 1.0000 - auc: 0.9875 - accuracy: 0.9486 - val_loss: 0.1394 - val_precision: 0.3810 - val_recall: 0.8889 - val_auc: 0.9308 

37/37 [==============================] - 18s 493ms/step - loss: 0.3853 - precision: 0.5128 - recall: 0.8571 - auc: 0.9016 - accuracy: 0.8281 - val_loss: 0.5159 - val_precision: 0.4717 - val_recall: 0.8475 - val_auc: 0.8444 - val_accuracy: 0.7782
Epoch 15/100
37/37 [==============================] - 18s 488ms/step - loss: 0.3695 - precision: 0.4947 - recall: 0.8952 - auc: 0.9088 - accuracy: 0.8169 - val_loss: 0.4590 - val_precision: 0.4608 - val_recall: 0.7966 - val_auc: 0.8528 - val_accuracy: 0.7713
Epoch 16/100
37/37 [==============================] - 18s 500ms/step - loss: 0.3753 - precision: 0.4909 - recall: 0.9000 - auc: 0.9063 - accuracy: 0.8144 - val_loss: 0.4813 - val_precision: 0.4571 - val_recall: 0.8136 - val_auc: 0.8631 - val_accuracy: 0.7679
Epoch 17/100
37/37 [==============================] - 18s 489ms/step - loss: 0.3472 - precision: 0.5148 - recall: 0.9095 - auc: 0.9214 - accuracy: 0.8298 - val_loss: 0.4322 - val_precision: 0.4902 - val_recall: 0.8475 - val_auc: 0.8694 

Epoch 46/100
37/37 [==============================] - 18s 485ms/step - loss: 0.2616 - precision: 0.5761 - recall: 0.9190 - auc: 0.9467 - accuracy: 0.8640 - val_loss: 0.3939 - val_precision: 0.5165 - val_recall: 0.7966 - val_auc: 0.8984 - val_accuracy: 0.8089
Epoch 47/100
37/37 [==============================] - 18s 488ms/step - loss: 0.2628 - precision: 0.5495 - recall: 0.9524 - auc: 0.9492 - accuracy: 0.8512 - val_loss: 0.4300 - val_precision: 0.5000 - val_recall: 0.7966 - val_auc: 0.8825 - val_accuracy: 0.7986
Epoch 48/100
37/37 [==============================] - 18s 483ms/step - loss: 0.2657 - precision: 0.5571 - recall: 0.9286 - auc: 0.9478 - accuracy: 0.8546 - val_loss: 0.3716 - val_precision: 0.5300 - val_recall: 0.8983 - val_auc: 0.9107 - val_accuracy: 0.8191
Epoch 49/100
37/37 [==============================] - 18s 495ms/step - loss: 0.2562 - precision: 0.5682 - recall: 0.9524 - auc: 0.9518 - accuracy: 0.8614 - val_loss: 0.3829 - val_precision: 0.5426 - val_recall: 0.8644 - val

98/98 [==============================] - 49s 504ms/step - loss: 0.1891 - precision: 0.4273 - recall: 0.9648 - auc: 0.9725 - accuracy: 0.8913 - val_loss: 0.3249 - val_precision: 0.3780 - val_recall: 0.8986 - val_auc: 0.9275 - val_accuracy: 0.8606
Epoch 25/100
25/25 [==============================] - 4s 168ms/step - loss: 0.2997 - precision: 0.4348 - recall: 0.8696 - auc: 0.9226 - accuracy: 0.8887
Loss: 0.29973697662353516, Precision: 0.43478259444236755, Recall: 0.8695651888847351, AUC: 0.9226477146148682, Accuracy: 0.8887467980384827
4 di 2581) Caricamento ticker MKSI                            


KeyboardInterrupt

